<a href="https://colab.research.google.com/github/imrahul1/Springboard/blob/master/Capstone/Copy_of_Copy_of_Modelling_Black_Friday_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#2.1 Get the train file
downloaded = drive.CreateFile({'id':'11dlq7CXZahRY3Y2KRsh2qLnO8F8h9cT5'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')

In [0]:
#2.2 Get the test file https://drive.google.com/file/d/1HIMB3baDDUupme2RYcr8pd0fZHwfAiw_/view?usp=sharing
downloaded2 = drive.CreateFile({'id':'1HIMB3baDDUupme2RYcr8pd0fZHwfAiw_'}) # replace the id with id of file you want to access
downloaded2.GetContentFile('test.csv')

<h3> Let's import the required packages first</h3>

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

<h3>Data Wrangling</h3>

In [0]:
test = pd.read_csv('test.csv')

In [8]:
test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
User_ID                       233599 non-null int64
Product_ID                    233599 non-null object
Gender                        233599 non-null object
Age                           233599 non-null object
Occupation                    233599 non-null int64
City_Category                 233599 non-null object
Stay_In_Current_City_Years    233599 non-null object
Marital_Status                233599 non-null int64
Product_Category_1            233599 non-null int64
Product_Category_2            161255 non-null float64
Product_Category_3            71037 non-null float64
dtypes: float64(2), int64(4), object(5)
memory usage: 19.6+ MB


In [10]:
train = pd.read_csv('train.csv')
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


As we can see many values in the Product categories are NaN. Let's have a deeper look into the structure of the DataFrame.

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            376430 non-null float64
Product_Category_3            166821 non-null float64
Purchase                      550068 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In the information we can see that product categories have lesser non-null entries than other columns.

In [12]:
train['Product_Category_1'].isnull().value_counts()

False    550068
Name: Product_Category_1, dtype: int64

Below we can see the Product_Category_2 column has many null observations.

In [13]:
train['Product_Category_2'].isnull().value_counts()

False    376430
True     173638
Name: Product_Category_2, dtype: int64

Here we can see that the non-null and null observations together make up the total entries

In [14]:
376430+173638

550068

In [15]:
train['Product_Category_3'].isnull().value_counts()

True     383247
False    166821
Name: Product_Category_3, dtype: int64

Even in the case of the column Product_Category_2 the null and non-null values together constitute the total number of observations.

In [16]:
383247+166821

550068

We can fill the null values as 0 because they were possibly not purchased during those transactions.

In [0]:
train.fillna(0, inplace=True)

In [0]:
test.fillna(0, inplace=True)

In [19]:
train.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,6.735436,3.841941,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,6.215492,6.250712,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,0.000000,0.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,0.000000,0.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,5.000000,0.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,14.000000,8.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            550068 non-null float64
Product_Category_3            550068 non-null float64
Purchase                      550068 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [21]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
User_ID                       233599 non-null int64
Product_ID                    233599 non-null object
Gender                        233599 non-null object
Age                           233599 non-null object
Occupation                    233599 non-null int64
City_Category                 233599 non-null object
Stay_In_Current_City_Years    233599 non-null object
Marital_Status                233599 non-null int64
Product_Category_1            233599 non-null int64
Product_Category_2            233599 non-null float64
Product_Category_3            233599 non-null float64
dtypes: float64(2), int64(4), object(5)
memory usage: 19.6+ MB


In [22]:
test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,0.0
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,0.0
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,0.0
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,0.0
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [23]:
test.Product_ID.dtype

dtype('O')

In [24]:
train.Product_ID.dtype

dtype('O')

<h2> Feature Engineering </h2>

In [0]:
train['Product_ID'] = train['Product_ID'].str.replace('P00', '')
test['Product_ID'] = test['Product_ID'].str.replace('P00','')

In [26]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,069042,F,0-17,10,A,2,0,3,0.0,0.0,8370
1,1000001,248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,087842,F,0-17,10,A,2,0,12,0.0,0.0,1422
3,1000001,085442,F,0-17,10,A,2,0,12,14.0,0.0,1057
4,1000002,285442,M,55+,16,C,4+,0,8,0.0,0.0,7969


In [27]:
train['Product_ID'].dtype

dtype('O')

<h4> Encoding the features to make them fit for modelling.
<br>Initializing Label Encoder</h4>

In [0]:
le = LabelEncoder()

In [29]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,069042,F,0-17,10,A,2,0,3,0.0,0.0,8370
1,1000001,248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,087842,F,0-17,10,A,2,0,12,0.0,0.0,1422
3,1000001,085442,F,0-17,10,A,2,0,12,14.0,0.0,1057
4,1000002,285442,M,55+,16,C,4+,0,8,0.0,0.0,7969


In [0]:
train['User_ID']= le.fit_transform(train['User_ID'])
test['User_ID'] = le.transform(test['User_ID'])

In [0]:
cols = ['Gender', 'Occupation', 'Age', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 
        'Product_Category_2','Product_Category_3']

In [0]:
for col in cols:
  train[col]=le.fit_transform(train[col])
  test[col] = le.transform(test[col])

In [33]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,0,069042,0,0,10,0,2,0,2,0,0,8370
1,0,248942,0,0,10,0,2,0,0,5,11,15200
2,0,087842,0,0,10,0,2,0,11,0,0,1422
3,0,085442,0,0,10,0,2,0,11,13,0,1057
4,1,285442,1,6,16,2,4,0,7,0,0,7969


In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null int64
Age                           550068 non-null int64
Occupation                    550068 non-null int64
City_Category                 550068 non-null int64
Stay_In_Current_City_Years    550068 non-null int64
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            550068 non-null int64
Product_Category_3            550068 non-null int64
Purchase                      550068 non-null int64
dtypes: int64(11), object(1)
memory usage: 50.4+ MB


<h2>Scaling and Normalizing </h2>

<h2>Scaling</h2>

In [0]:
num_col = ['Age', 'Occupation', 'Stay_In_Current_City_Years', 'Product_Category_1', 
           'Product_Category_2', 'Product_Category_3']

In [0]:
scale = StandardScaler()

In [0]:
for col in num_col:
    train[col] = scale.fit_transform(train[col].values.reshape(-1, 1))
    test[col] = scale.transform(test[col].values.reshape(-1, 1))

<h2> Modelling</h2>

In [0]:
X = train.drop(['Product_ID','Purchase'], axis=1)
y = train['Purchase']

<h4>Calling the train test split</h4>

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.30, random_state=0, shuffle=True)
X_test = test.drop(['Product_ID'], axis=1)

<h4>Initializing Linear Regression </h4>

In [0]:
linr=LinearRegression()

In [41]:
linr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
y_pred = linr.predict(X_val)

In [43]:
linr.score(X_val,y_val)

0.15076672080655062

<h4>Scoring the predictions based on RMSE</h4>

In [44]:
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = print(np.sqrt(mean_squared_error(y_pred, y_val)))

4634.693700800083


<h4>Importing and initializing Decision Tree Regressor</h4>

In [0]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()

In [46]:
dtr.fit(X_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
y_pred_rd=dtr.predict(X_val)

<h4>Scoring Decision Tree</h4>

In [48]:
dtr.score(X_val,y_val)

0.521739624157582

In [49]:
print(np.sqrt(mean_squared_error(y_val, y_pred_rd)))

3478.0813504127955


<h4>Random Forest Regressor </h4>

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
rfr = RandomForestRegressor()

In [52]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
y_pred_rfr = rfr.predict(X_val)

In [54]:
rfr.score(X_val, y_val)

0.6480340304025614

In [55]:
print(np.sqrt(mean_squared_error(y_val, y_pred_rfr)))

2983.7186768976867


In [0]:
predicted = rfr.predict(X_test)

In [0]:
submission = pd.read_csv('test.csv')

In [0]:
submission['Purchase'] = predicted

In [0]:
submission.to_csv('Black Friday Submission.csv', index=False)

In [67]:
submission.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN,14600.615000
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN,8729.890000
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN,8692.126821
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN,2519.481333
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0,2879.310000


In [0]:
from google.colab import files
files.download('Black Friday Submission.csv')

RMSE at Hackathon is  2991.5730247323 

<h2>Cross Validation</h2>

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
cv_results = cross_val_score(rfr, X, y, cv=5)

In [70]:
print(cv_results)

[0.63772531 0.63955309 0.64216476 0.64006601 0.66830698]


In [71]:
np.mean(cv_results)

0.6455632270326326

In [72]:
rfr.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

<h2>Hypertuning</h2>
<br><h4>Default number of estimators in Random Forest Regressor is 10 and maximum depth is None</h4>

In [73]:
from sklearn.model_selection import GridSearchCV

param_grid={
    'max_depth': [3, None],
     'n_estimators': (3,5,10)}
model = GridSearchCV(rfr, param_grid, cv=5)
model.fit(X,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [74]:
model.best_params_

{'max_depth': None, 'n_estimators': 10}